In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# Data Preperation

In [4]:
data=pd.read_excel("data/online_retail_II.xlsx",
                   sheet_name="Year 2010-2011")

In [5]:
data.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55000,17850.00000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75000,17850.00000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom


In [6]:
# let us drop duplicates
filteredData = data[['Country', 'Customer ID']].drop_duplicates()

In [7]:
filteredData.Country.value_counts()

United Kingdom        3951
Germany                 95
France                  88
Spain                   31
Belgium                 25
                      ... 
Lithuania                1
Brazil                   1
Iceland                  1
RSA                      1
European Community       1
Name: Country, Length: 38, dtype: int64

In [8]:
# Filter only UK data
uk_data = data[data['Country'] == 'United Kingdom']
print(uk_data.describe())

          Quantity        Price  Customer ID
count 495478.00000 495478.00000 361878.00000
mean       8.60549      4.53242  15547.87137
std      227.58876     99.31544   1594.40259
min   -80995.00000 -11062.06000  12346.00000
25%        1.00000      1.25000  14194.00000
50%        3.00000      2.10000  15514.00000
75%       10.00000      4.13000  16931.00000
max    80995.00000  38970.00000  18287.00000


In [9]:
# Removing negative quantity values
uk_data = uk_data[uk_data['Quantity'] > 0]
print(uk_data.describe())

          Quantity        Price  Customer ID
count 486286.00000 486286.00000 354345.00000
mean       9.73464      3.79498  15552.43622
std      163.26243     41.27116   1594.54603
min        1.00000 -11062.06000  12346.00000
25%        1.00000      1.25000  14194.00000
50%        3.00000      2.10000  15522.00000
75%       10.00000      4.13000  16931.00000
max    80995.00000  13541.33000  18287.00000


In [10]:
uk_data.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country'],
      dtype='object')

In [11]:
# use only required columns for CLTV
uk_data = uk_data[['Customer ID', 'InvoiceDate', 'Invoice', 'Quantity', 'Price']]

In [12]:
# calculate total purchase
uk_data['TotalPurchase'] = uk_data['Quantity'] * uk_data['Price']


In [13]:
# perform aggregate operations
uk_data_grp = uk_data.groupby('Customer ID').agg({
    'InvoiceDate': lambda date: (date.max() - date.min()).days,
    'Invoice': lambda num: len(num),
    'Quantity': lambda quantity: quantity.sum(),
    'TotalPurchase': lambda total_pur: total_pur.sum()
})

uk_data_grp.head()

,InvoiceDate,Invoice,Quantity,TotalPurchase
Customer ID,,,,
12346.00000,0,1,74215,77183.60000
12747.00000,366,103,1275,4196.01000
12748.00000,372,4596,25748,33719.73000
12749.00000,209,199,1471,4090.88000
12820.00000,323,59,722,942.34000


CLTV = ((Average Order Value x Purchase Frequency)/Churn Rate) x Profit margin.<br>
Customer Value = Average Order Value * Purchase Frequency


In [14]:
# 1. Calculate Average Order Value
uk_data_grp['avg_order_val'] = uk_data_grp['TotalPurchase'] / uk_data_grp['Quantity']

In [15]:
uk_data_grp.shape

(3921, 5)

In [16]:
# 2. Calculate Purchase Frequency
purchase_frequency = sum(uk_data_grp['Invoice'])/uk_data_grp.shape[0]

In [17]:
purchase_frequency

90.37107880642694

In [18]:
# 3. Calculate Repeat and Churn Rate
repeat_rate=uk_data_grp[uk_data_grp['Invoice'] > 1].shape[0]/uk_data_grp.shape[0]
churn_rate = 1 - repeat_rate

print(purchase_frequency, repeat_rate, churn_rate)


90.37107880642694 0.9818923743942872 0.018107625605712774


In [19]:
# 4. Calculate Profit Margin
# Calculate Profit margin assuming gain of 5%

uk_data_grp['profit_margin'] = uk_data_grp['TotalPurchase'] * 0.05


In [20]:
# 5. Calculate Customer Lifetime Value
uk_data_grp['cust_lifetime_value'] = ((uk_data_grp['avg_order_val'] * purchase_frequency)/churn_rate) * \
                                     uk_data_grp['profit_margin']

uk_data_grp.head()


,InvoiceDate,Invoice,Quantity,TotalPurchase,avg_order_val,profit_margin,cust_lifetime_value
Customer ID,,,,,,,
12346.00000,0,1,74215,77183.60000,1.04000,3859.18000,20030709.61386
12747.00000,366,103,1275,4196.01000,3.29099,209.80050,3445885.23291
12748.00000,372,4596,25748,33719.73000,1.30961,1685.98650,11019519.07865
12749.00000,209,199,1471,4090.88000,2.78102,204.54400,2838956.72483
12820.00000,323,59,722,942.34000,1.30518,47.11700,306913.51954


In [29]:
uk_data_grp['cust_lifetime_value'].describe()

count         3921.00000
mean       2097860.63190
std       55264929.79530
min              0.00000
25%         130768.48527
50%         306336.95624
75%         749382.94719
max     3285923908.63653
Name: cust_lifetime_value, dtype: float64

In [30]:
uk_data_grp[uk_data_grp['cust_lifetime_value']==0]

,InvoiceDate,Invoice,Quantity,TotalPurchase,avg_order_val,profit_margin,cust_lifetime_value
Customer ID,,,,,,,
13256.00000,0,1,12540,0.00000,0.00000,0.00000,0.00000


# Prediction Model for CLTV

In [21]:
uk_data.head(2)

,Customer ID,InvoiceDate,Invoice,Quantity,Price,TotalPurchase
0,17850.00000,2010-12-01 08:26:00,536365,6,2.55000,15.30000
1,17850.00000,2010-12-01 08:26:00,536365,6,3.39000,20.34000


In [22]:
uk_data['month_yr'] = uk_data['InvoiceDate'].apply(lambda x: x.strftime('%b-%Y'))
uk_data.head(2)

,Customer ID,InvoiceDate,Invoice,Quantity,Price,TotalPurchase,month_yr
0,17850.00000,2010-12-01 08:26:00,536365,6,2.55000,15.30000,Dec-2010
1,17850.00000,2010-12-01 08:26:00,536365,6,3.39000,20.34000,Dec-2010


In [25]:
sale = uk_data.pivot_table(index=['Customer ID'], columns=['month_yr'], values='TotalPurchase', aggfunc='sum', fill_value=0).reset_index()

sale.head()

month_yr,Customer ID,Apr-2011,Aug-2011,Dec-2010,Dec-2011,Feb-2011,Jan-2011,Jul-2011,Jun-2011,Mar-2011,May-2011,Nov-2011,Oct-2011,Sep-2011
0,12346.00000,0.00000,0.00000,0.00000,0.00000,0.00000,77183.60000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,12747.00000,0.00000,301.70000,706.27000,438.50000,0.00000,303.04000,0.00000,376.30000,310.78000,771.31000,312.73000,675.38000,0.00000
2,12748.00000,1100.37000,898.24000,4228.13000,1070.27000,389.64000,418.77000,1113.27000,2006.26000,1179.37000,2234.50000,10639.23000,2292.84000,6148.84000
3,12749.00000,0.00000,1896.13000,0.00000,763.06000,0.00000,0.00000,0.00000,0.00000,0.00000,859.10000,572.59000,0.00000,0.00000
4,12820.00000,0.00000,0.00000,0.00000,210.35000,0.00000,170.46000,0.00000,0.00000,0.00000,0.00000,0.00000,343.76000,217.77000


In [26]:
sale.iloc[:, 1:]

month_yr,Apr-2011,Aug-2011,Dec-2010,Dec-2011,Feb-2011,Jan-2011,Jul-2011,Jun-2011,Mar-2011,May-2011,Nov-2011,Oct-2011,Sep-2011
0,0.00000,0.00000,0.00000,0.00000,0.00000,77183.60000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,0.00000,301.70000,706.27000,438.50000,0.00000,303.04000,0.00000,376.30000,310.78000,771.31000,312.73000,675.38000,0.00000
2,1100.37000,898.24000,4228.13000,1070.27000,389.64000,418.77000,1113.27000,2006.26000,1179.37000,2234.50000,10639.23000,2292.84000,6148.84000
3,0.00000,1896.13000,0.00000,763.06000,0.00000,0.00000,0.00000,0.00000,0.00000,859.10000,572.59000,0.00000,0.00000
4,0.00000,0.00000,0.00000,210.35000,0.00000,170.46000,0.00000,0.00000,0.00000,0.00000,0.00000,343.76000,217.77000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3916,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,180.60000,0.00000,0.00000,0.00000,0.00000
3917,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,80.82000,0.00000,0.00000,0.00000,0.00000,0.00000
3918,0.00000,100.21000,0.00000,77.84000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3919,117.68000,0.00000,0.00000,208.00000,102.90000,215.00000,143.19000,307.53000,0.00000,99.47000,651.56000,114.65000,134.90000


In [39]:
sale['CLV'] = sale.iloc[:, 2:].sum(axis=1)

sale.head()

month_yr,Customer ID,Apr-2011,Aug-2011,Dec-2010,Dec-2011,Feb-2011,Jan-2011,Jul-2011,Jun-2011,Mar-2011,May-2011,Nov-2011,Oct-2011,Sep-2011,CLV
0,12346.00000,0.00000,0.00000,0.00000,0.00000,0.00000,77183.60000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,77183.60000
1,12747.00000,0.00000,301.70000,706.27000,438.50000,0.00000,303.04000,0.00000,376.30000,310.78000,771.31000,312.73000,675.38000,0.00000,4196.01000
2,12748.00000,1100.37000,898.24000,4228.13000,1070.27000,389.64000,418.77000,1113.27000,2006.26000,1179.37000,2234.50000,10639.23000,2292.84000,6148.84000,32619.36000
3,12749.00000,0.00000,1896.13000,0.00000,763.06000,0.00000,0.00000,0.00000,0.00000,0.00000,859.10000,572.59000,0.00000,0.00000,4090.88000
4,12820.00000,0.00000,0.00000,0.00000,210.35000,0.00000,170.46000,0.00000,0.00000,0.00000,0.00000,0.00000,343.76000,217.77000,942.34000


In [40]:
X=sale[['Dec-2011','Nov-2011', 'Oct-2011','Sep-2011','Aug-2011','Jul-2011']]
y=sale[['CLV']]

In [61]:
pd.concat([X,y], axis=1)

month_yr,Dec-2011,Nov-2011,Oct-2011,Sep-2011,Aug-2011,Jul-2011,CLV
0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,77183.60000
1,438.50000,312.73000,675.38000,0.00000,301.70000,0.00000,4196.01000
2,1070.27000,10639.23000,2292.84000,6148.84000,898.24000,1113.27000,32619.36000
3,763.06000,572.59000,0.00000,0.00000,1896.13000,0.00000,4090.88000
4,210.35000,0.00000,343.76000,217.77000,0.00000,0.00000,942.34000
...,...,...,...,...,...,...,...
3916,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,180.60000
3917,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,80.82000
3918,77.84000,0.00000,0.00000,0.00000,100.21000,0.00000,178.05000
3919,208.00000,651.56000,114.65000,134.90000,0.00000,143.19000,1977.20000


In [41]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [42]:
# Train and predict the model
LR = LinearRegression()
LR.fit(X_train, y_train)

y_pred_score = LR.score(X_test, y_test)
print("Prediction score is: {}".format(y_pred_score))

y_pred = LR.predict(X_test)

Prediction score is: 0.9078398355505256


In [43]:
# Check R-Squared value
print("R-Squared: {}".format(metrics.r2_score(y_test, y_pred)))

R-Squared: 0.9078398355505256
